In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
import plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
#init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Extra options
# pd.options.display.max_rows = 30
# pd.options.display.max_columns = 25

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import os
from IPython.display import Image, display, HTML
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [27]:
## Copy paste

%run login.py

loginquery = f"""
mutation {{
  logIn(
      email:\"{login}\",
      password:\"{pwd}\") {{
    jwt {{
      token
      exp
    }}
  }}
}}
"""

import requests
url = 'https://api.numina.co/graphql'

mylogin = requests.post(url, json={'query': loginquery})

token = mylogin.json()['data']['logIn']['jwt']['token']

device_ids = ['SWLSANDBOX1', 'SWLSANDBOX2', 'SWLSANDBOX3']

In [24]:
def text_explaination(camera, zone, function, procedure):
    print('This {0} behavior zone is under {1} camera. It servers for {2}.\nWe will explore this zone through {3}.'
          .format(zone, camera, function, procedure))

In [23]:
fdir = 'BehaviorZoneImage/'
@interact
def show_images(Zone=os.listdir(fdir)):
    display(Image(fdir+Zone))
    if (Zone == 'Outside-ChairZone.png'):
            text_explaination('Outside', 'ChairZone', 'seating or taking a break for pedestrians', 'its desired spots')
    if (Zone == 'Outside-PathZone.png'):
            text_explaination('Outside', 'PathZone', 'path for pedestrians, vehicles, bikes', 'desired lines and spots for pedastrains and traffics')
    if (Zone == 'Streetscape-PathZone.png'):
            text_explaination('Streetscape', 'PathZone', 'path for pedestrians', 'desired line and spots for pedastrains')
    if (Zone == 'Streetscape-ChairZone.png'):
            text_explaination('Streetscape', 'ChairZone', 'activity seating or taking a break for pedestrians', 'its desired spots')
    if (Zone == 'Streetscape-ActivityZone.png'):
            text_explaination('Streetscape', 'ActivityZone', 'freely walking for pedastrains or holding activities', 'its desired spots, lines')
    if (Zone == 'UnderRaincoat-TrafficZone.png'):
            text_explaination('UnderRaincoat', 'TrafficZone', 'path for pedestrians, vehicles, bikes', 'its desired spots, lines for pedastrains and traffics')
    if (Zone == 'UnderRaincoat-ChairZone.png'):
            text_explaination('UnderRaincoat', 'ChairZone', 'factivity seating or taking a break for pedestrians', 'its desired spots')
    if (Zone == 'UnderRaincoat-ActivityZone.png'):
            text_explaination('UnderRaincoat', 'ActivityZone', 'freely walking for pedastrains', 'its desired spots, lines')

interactive(children=(Dropdown(description='Zone', options=('Outside-ChairZone.png', 'Outside-PathZone.png', '…

**This is for keeping track of my work**
1. Get zones ids, store them
2. Zone heatmap based on day of all zones, store them in csv
3. Interaction for path zone: we do not use rectangle, because it has angle, so visualize, interative part, date, quantile, types

In [25]:
## copy and paste
def get_zones(device_id):
    query_zones = """
    query {{
      behaviorZones (
        serialnos: "{0}"
        ) {{
        count
        edges {{
          node {{
            rawId
            text
          }}
        }}
      }}
    }}
    """.format(device_id)
    
    zones = requests.post(url, json={'query': query_zones}, headers = {'Authorization':token})
    
    df = pd.DataFrame([x['node'] for x in zones.json()['data']['behaviorZones']['edges']])
    df['device'] = device_id
    
    return df

In [28]:
zones_df = pd.concat([get_zones(device_ids[i]) for i in range(3)])

In [37]:
## get our gourp zone
x_zones_df = zones_df[zones_df['text'].apply(lambda x: True if (x is not None and x[0:2] == 'x-') else False)]
x_zones_id = list(x_zones_df['rawId'])

In [126]:
## please do not input mutiple zoneId or obj, you can always use a for loop to construct a dataframe with multiple objects
def zone_heatmap_query_gen(startTime: str, endTime: str, zoneId: int, obj =str):
    zoneId = str(zoneId)
    heatmap_query = """
query {{
    zoneHeatmaps(
    zoneIds: [{0}],
    startTime:"{1}",
    endTime:"{2}",
    objClasses:["{3}"],
    timezone:"America/New_York") {{
    edges {{
        node {{
        time
        objClass
        heatmap
      }}
    }}
  }}
}}
""".format(zoneId, startTime, endTime, obj)
    return heatmap_query

In [127]:
from datetime import timedelta, datetime
from dateutil.relativedelta import relativedelta
import calendar
START_DATE = datetime(2019, 2, 20, 0, 0, 0)
END_DATE = datetime(2019, 2, 25, 0, 0, 0)
time_delta = relativedelta(days = +1)

In [128]:
##heatmap_data = requests.post(url, json={'query': zone_heatmap_query_gen(start_time_str, end_time_str, '14895', 'pedestrian')}, 
##                         headers = {'Authorization':token})
## This is an example in case I forgot

In [162]:
## start_times and end_times are required to have same length
def get_zoneheatmap_data(zoneid: int, obj: str, start_times: list, end_times: list):
    zone_heatmap_df = pd.DataFrame(columns = ['startTime', 'endTime', 'heatMap', 'obj'])
    i = 0
    while i < len(start_times):
        heatmap_data = requests.post(url, json={'query': zone_heatmap_query_gen(start_times[i].strftime('%Y-%m-%dT%H:%M:%S'), 
                                                                                end_times[i].strftime('%Y-%m-%dT%H:%M:%S'), zoneid, obj)}, 
                                                                                headers = {'Authorization':token})
        heatmap_json = heatmap_data.json()
        if heatmap_json['data']:
            if 'zoneHeatmaps' in heatmap_json['data']:
                heatmap = heatmap_json['data']['zoneHeatmaps']['edges'][0]['node']['heatmap']
                temp_df = pd.DataFrame({"startTime":start_times[i], "endTime":end_times[i], 'heatMap':heatmap})
                zone_heatmap_df = zone_heatmap_df.append(temp_df, ignore_index = True)
        i = i + 1
    return zone_heatmap_df

In [163]:
def generate_consecutive_times(start_time: datetime, end_time: datetime, interval: relativedelta):
    ## the first element in the list are the start times
    time = [[], []]
    current_time = start_time
    while current_time < end_time:
        time[0].append(current_time)
        time[1].append(current_time + interval)
        current_time = current_time + interval
    return time

In [164]:
time = generate_consecutive_times(START_DATE, END_DATE, time_delta)

In [ ]:
get_zoneheatmap_data(x_zones_id[0], 'pedestrian', time[0], time[1])

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


